# CNN model
This notebook trains the CNN model.

**Trains on:** SemEval (2019)

First we need to install the required packages.

In [ ]:
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive

# Mount drive for loading the data
drive.mount('/content/drive')

import sys
sys.path.append("drive/MyDrive/Colab Notebooks")

import tensorflow
from cnn import CNN
from reader import Reader
from sklearn.metrics import classification_report, confusion_matrix 
import numpy as np
from tensorflow.keras.utils import to_categorical
import html
from keras.layers import TextVectorization
import tensorflow_probability
import pickle

FILENAME = "drive/MyDrive/Colab Notebooks/data/sem_eval_all.pkl"
EPOCHS = 10
BATCH_SIZE = 128
NUM_CLASSES = 2
LEARN_RATE = 0.01
EMBED_SIZE = 100
LOSS_TYPE = "logits"
MAX_LEN = 250
MAX_TOKENS = 20000

Mounted at /content/drive


## Read and split the dataset

In [ ]:
reader = Reader(filename=FILENAME)
X, y = reader.load()

mapping = {'hate': 1, 'none': 0}
y = [mapping[b] for b in y]

X_train, X_val, X_test, y_train, y_val, y_test = reader.split_with_validation(X, y)

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
X_val = np.asarray(X_val)

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)
y_val = to_categorical(y_val, num_classes=NUM_CLASSES)

## Setup the TextVectorization layer

In [ ]:
text_vectorizer = TextVectorization(
    max_tokens=MAX_TOKENS,
    output_mode='int',
    output_sequence_length=MAX_LEN)
text_vectorizer.adapt(X_train)

vocab = text_vectorizer.get_vocabulary()

## Setup the model

In [ ]:
model = CNN(max_len=MAX_LEN,
          num_classes=NUM_CLASSES, 
          batch_size=BATCH_SIZE, 
          epochs=EPOCHS, 
          embed_size=EMBED_SIZE, 
          vocab_len=MAX_TOKENS,
          loss_type=LOSS_TYPE,
          save_model=True,
          save_path="drive/MyDrive/Colab Notebooks/output/cnn-semeval",
          checkpoint_path="drive/MyDrive/Colab Notebooks/output/cnn-semeval.ckpt",
          text_vectorizer=text_vectorizer)

## Training

In [ ]:
model.fit(X_train, y_train)

Epoch 1/10
67/68 [============================>.] - ETA: 0s - loss: 1.7835 - accuracy: 0.5756
Epoch 1: saving model to drive/MyDrive/Colab Notebooks/output/cnn-semeval.ckpt
68/68 [==============================] - 6s 72ms/step - loss: 1.7767 - accuracy: 0.5753
Epoch 2/10
67/68 [============================>.] - ETA: 0s - loss: 0.7164 - accuracy: 0.6053
Epoch 2: saving model to drive/MyDrive/Colab Notebooks/output/cnn-semeval.ckpt
68/68 [==============================] - 5s 73ms/step - loss: 0.7158 - accuracy: 0.6058
Epoch 3/10
67/68 [============================>.] - ETA: 0s - loss: 0.6168 - accuracy: 0.6874
Epoch 3: saving model to drive/MyDrive/Colab Notebooks/output/cnn-semeval.ckpt
68/68 [==============================] - 5s 73ms/step - loss: 0.6164 - accuracy: 0.6878
Epoch 4/10
67/68 [============================>.] - ETA: 0s - loss: 0.5363 - accuracy: 0.7629
Epoch 4: saving model to drive/MyDrive/Colab Notebooks/output/cnn-semeval.ckpt
68/68 [==============================] - 5s 

In [ ]:
model_predictions = model.cnn.predict(X_test)
predicted_classes = np.argmax(model_predictions, axis=1)
actual_classes = np.argmax(y_test, axis=1)

print(confusion_matrix(actual_classes, predicted_classes))
print(classification_report(actual_classes, predicted_classes))

[[532 164]
 [196 308]]
              precision    recall  f1-score   support

           0       0.73      0.76      0.75       696
           1       0.65      0.61      0.63       504

    accuracy                           0.70      1200
   macro avg       0.69      0.69      0.69      1200
weighted avg       0.70      0.70      0.70      1200



## Waseem predictions calibration
We use temperature scaling to calibrate the model by finding the optimal T value on the validation set.



In [ ]:
# Implementation from: https://github.com/ritun16/Machine_Learning_short_projecct/blob/master/Neural_Network_Calibration/model_calibration.ipynb
# ECE result before calibration
num_bins = 50
model_predictions = model.cnn.predict(X_val)
model_actual_classes = np.argmax(y_val, axis=1)
model_labels = tensorflow.convert_to_tensor(model_actual_classes, dtype=tensorflow.int32, name='labels_true')
model_logits = tensorflow.convert_to_tensor(model_predictions, dtype=tensorflow.float32, name='logits')
tensorflow_probability.stats.expected_calibration_error(num_bins=num_bins, 
                                     logits=model_logits, 
                                     labels_true=model_labels)

<tf.Tensor: shape=(), dtype=float32, numpy=0.14167416>

In [ ]:
# Temperature Scaling
temp = tensorflow.Variable(initial_value=1.0, trainable=True, dtype=tensorflow.float32) 

def compute_loss():
    pred_divisions = tensorflow.math.divide(model_predictions, temp)
    loss = tensorflow.reduce_mean(tensorflow.nn.softmax_cross_entropy_with_logits(\
                                tensorflow.convert_to_tensor(tensorflow.keras.utils.to_categorical(model_actual_classes)), pred_divisions))
    return loss

optimizer = tensorflow.optimizers.Adam(learning_rate=0.01)

print('Temperature Initial value: {}'.format(temp.numpy()))

for i in range(300):
    opts = optimizer.minimize(compute_loss, var_list=[temp])


print('Temperature Final value: {}'.format(temp.numpy()))

Temperature Initial value: 1.0
Temperature Final value: 2.216935634613037


In [ ]:
# ECE result after calibration
pred_divisions = tensorflow.math.divide(model_predictions, temp)
num_bins = 50
labels_true = tensorflow.convert_to_tensor(model_actual_classes, dtype=tensorflow.int32, name='labels_true')
logits = tensorflow.convert_to_tensor(pred_divisions, dtype=tensorflow.float32, name='logits')
tensorflow_probability.stats.expected_calibration_error(num_bins=num_bins, 
                                     logits=logits, 
                                     labels_true=labels_true)

<tf.Tensor: shape=(), dtype=float32, numpy=0.04484203>